In [ ]:
# pip install spacy
# pip install spacy_transformers

# Create a spacy object


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import sys, fitz

In [ ]:
pip list

In [ ]:
import warnings
warnings.filterwarnings("ignore", category = UserWarning)

In [ ]:
cv_data = json.load(open('E:/PROJECTS/Resume-Parser-And-Mathcer/train_data.json','r'))

In [ ]:
cv_data

In [ ]:

len(cv_data)

In [ ]:
!python -m spacy init fill-config E:/PROJECTS/Resume-Parser-And-Mathcer/base_config.cfg config.cfg

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en') #  load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        annot = annot['entities']
        ents=[]
        entity_indices=[]
        for start, end, label in annot:
            # add character indexes
            skip_entity=False
            for idx in range(start,end):
                if idx in entity_indices:
                    skip_entity=True
                    break
            if skip_entity==True:
                continue
                
            entity_indices=entity_indices+list(range(start,end))
            
            try:
                span = doc.char_span(start, end, label=label, alignment_mode = 'strict')
            except:
                continue
            if span is None:
                err_data = str([start, end]) + "  " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)
            
        try:
            doc.ents=ents
            db.add(doc)
        except:
            pass
    return db

# db=get_spacy_doc(data)
# db.to_disk("./train.spacy")
    

In [ ]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(cv_data, test_size = 0.3)

In [ ]:
len(train), len(test)

In [ ]:
file = open('error.txt', 'w', encoding='utf-8')
# with open('filename', 'w', encoding='utf-8') as f:

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [ ]:
len(db.tokens)

# TRAINING AND LOADING THE MODEL

In [ ]:
# !python -m spacy train E:/PROJECTS/Resume-Parser-And-Mathcer/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy

# Model Test

In [ ]:
nlp = spacy.load('E:/PROJECTS/selfTry/output/model-best')

In [ ]:
doc = nlp('My name is Nikil Saini, I am an Intern at Amazon, I have 3 months of experience')

In [ ]:
doc

In [ ]:
for ent in doc.ents:
    
    print(ent.text, "  --->>>>", ent.label_)

In [ ]:
# !pip install PyMuPDF

In [ ]:
import sys, fitz

In [ ]:
fname = 'E:/PROJECTS/selfTry/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [ ]:
doc

In [ ]:
text = " "
for page in doc:
    text = text + str(page.get_text())
text

In [ ]:
text = text.strip()
text

In [ ]:
text = ' '.join(text.split())

In [ ]:
# for i,_ in text:
#     doc = nlp(i)
#     print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
from pdfminer.high_level import extract_text
def text_from_pdf(pdf):
    return extract_text(pdf)

# Taking resume as input
resume = text_from_pdf("Pulkit_Saxena_resume.pdf")


# Removing the punctuations
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~•'''
for i in resume:
    if i in punctuations:
        resume = resume.replace(i, "")
        
# Obtaing cleaned text from pdf
final_text = str.join(" ", resume.splitlines())

from spacy import displacy
doc = nlp(final_text)
ans = [(ent.text, ent.label_) for ent in doc.ents]
displacy.render(doc,style='ent')

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, "  --->>>>", ent.label_)

# Similarity Match

In [ ]:
pref_skillS = 30
pref_degreeS = 10
pref_companiesS = 20

matched_skill=0
matched_degree=0
matched_companies=0
for i in ans:
    if i[1]=='Companies worked at':
        if i[0] in company_string:
            matched_companies+=1
    if i[1]=='Degree':
        if i[0] in degree_string:
            matched_degree+=1
    if i[1]=='Skills':
        if i[0].lower() in skill_string:
            matched_skill+=1

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity([[matched_skill,matched_degree,matched_companies]],[[pref_skillS,pref_degreeS,pref_companiesS]])
print(f"The input resume matches {similarity[0][0]*100:.2f} percent with the required job position.")